In [1]:
import requests
import folium

# Fetch earthquake data
earthquake_url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_week.geojson'
earthquake_data = requests.get(earthquake_url).json()

# Fetch tectonic plates data
tectonic_plates_url = 'https://raw.githubusercontent.com/fraxen/tectonicplates/master/GeoJSON/PB2002_boundaries.json'
tectonic_plates_data = requests.get(tectonic_plates_url).json()

In [2]:
# Create a base map
m = folium.Map(location=[0, 0], zoom_start=2)

# Add base maps (OpenStreetMap, Stamen Terrain, etc.)
folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('Stamen Toner').add_to(m)
folium.TileLayer('Stamen Watercolor').add_to(m)
folium.TileLayer('CartoDB positron').add_to(m)

# Function to add earthquake markers
def add_earthquake_layer(data):
    earthquake_layer = folium.FeatureGroup(name='Earthquakes')
    for feature in data['features']:
        coord = feature['geometry']['coordinates']
        folium.CircleMarker(
            location=[coord[1], coord[0]],
            radius=coord[2] / 100,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7,
            popup=folium.Popup(f"Magnitude: {feature['properties']['mag']}", max_width=300)
        ).add_to(earthquake_layer)
    return earthquake_layer

# Function to add tectonic plate boundaries
def add_tectonic_plates_layer(data):
    plates_layer = folium.FeatureGroup(name='Tectonic Plates')
    for feature in data['features']:
        coordinates = feature['geometry']['coordinates']
        folium.PolyLine(
            locations=[[coord[1], coord[0]] for coord in coordinates],
            color='blue',
            weight=2
        ).add_to(plates_layer)
    return plates_layer

# Add layers to the map
earthquake_layer = add_earthquake_layer(earthquake_data)
tectonic_plates_layer = add_tectonic_plates_layer(tectonic_plates_data)

earthquake_layer.add_to(m)
tectonic_plates_layer.add_to(m)

# Add layer control
folium.LayerControl().add_to(m)


In [5]:
m.save('map.html')
m
